### 🧱 Installing Python Packages

In [1]:
%pip install -U pymupdf sentence-transformers langchain qdrant-client llama-cpp-python numpy==1.26.4


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import packages

In [2]:
import os
import pymupdf

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from llama_cpp import Llama
from dotenv import load_dotenv
import numpy as np

### 📄 Load and read the PDF content

In [3]:
def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = "./pdf_document/StopSmoking.pdf"
if os.path.exists(pdf_path):
    print(f"File found at: {pdf_path}")
else:
    print(f"File not found at: {pdf_path}")

raw_text = extract_text_from_pdf(pdf_path)
print("Texto extraído totalmente:") # print("Texto extraído (primeros 500 caracteres):")
print(raw_text) # print(raw_text[:500])

File found at: ./pdf_document/StopSmoking.pdf
Texto extraído totalmente:
1 
 
 
 
 
 
MANUAL 
EASYWAY TO STOP SMOKING 
EL MÉTODO FÁCIL PARA DEJAR DE FUMAR. 
 
 
2 
 
ALLEN CARR’S EASY WAY TO STOP SMOKING THERAPY – PRIMERA SESIÓN 
PARTE 1  
 
A. GENERALES 
 
(Sub-títulos en negrita, junto con las palabras entre paréntesis y en cursiva no son 
parte del diálogo, pero a modo de explicación al terapeuta. En la práctica 
sobre75% de cada sesión consiste en el esqueleto básico estipulado en el manual 
a continuación. El 25% restante se compone de puntos periféricos o anécdotas 
contadas a hacer frente a cualquier cuestión específica que pueda surgir. 
 
El diálogo escrito en negritas significa que debe hacerse hincapié, y no 
necesariamente significa que debe uno gritar o dar puñetazos al aire. Diferentes 
terapeutas adoptan diversas técnicas. Personalmente creo que la técnica más 
eficaz es pausar un poco y luego decir las palabras lentamente y con énfasis y 
luego volver a pausar para dar 

### ✂️ Text Chunking

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

chunks = splitter.split_text(raw_text)
print(f"Total de chunks generados: {len(chunks)}")

Total de chunks generados: 463


### 🧬 Load a pretrained Sentence Transformer model & Create embeddings

In [5]:
# The Sentence Transformers framework contains many embedding models.
# I’ll take all-MiniLM-L6-v2 as it has a good balance between speed and embedding quality.
# https://www.sbert.net/#
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
embedder = SentenceTransformer("all-MiniLM-L6-v2")

chunk_embeddings = embedder.encode(chunks, convert_to_tensor=False)
print("Embeddings generados.")
print(chunk_embeddings.shape)

Embeddings generados.
(463, 384)


### 🧠 Store the obtained Embedding in the Vector Database

In [6]:
# QDRANT_URL = os.getenv("QDRANT_URL")
# QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
# print(f"Qdrant URL: {QDRANT_URL}")
# print(f"Qdrant API Key: {QDRANT_API_KEY}")
#client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# client = QdrantClient(":memory:")
client = QdrantClient("http://localhost:6333")


### 📦 Naming the Collection

In [7]:
# name_of_collection="stop_smoking"
name_of_collection="allen_carr"

### 💡 Create a collection

In [15]:
client.create_collection(
    collection_name=name_of_collection,
    vectors_config=models.VectorParams(
        size=embedder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

### 🆙 Upload Data to Collection
The database is uploading documents to the collection. This will give each record an id and a payload. The payload is just the metadata from the dataset.

In [16]:
client.upload_points(
    collection_name=name_of_collection,
    points=[
        models.PointStruct(
            id=i,
            vector=embedding.tolist(),
            payload={
                "text": chunk,
                "source": pdf_path,
                "chunk_index": i
            } 
        )
        for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings))
    ],
)

### 🔍 Context Retrieval Function

In [19]:
def retrieve_context(query, k=3):
    query_vec = embedder.encode([query])[0].tolist()

    results = client.query_points(
        collection_name=name_of_collection,
        query=query_vec,
        limit=k
    ).points

    return [hit.payload["text"] for hit in results]

### 🧠 Building a Prompt

In [20]:
def build_prompt(query):
    context_chunks = retrieve_context(query)
    context = "\n---\n".join(context_chunks)
    prompt = f"""
Responde con base en el siguiente contexto del negocio:

-------------------------------------------------------

{context}

-------------------------------------------------------

La pregunta fue: {query}
"""
    return prompt

### 🧪 Testing

In [21]:
query = "¿Cuál es el EFECTO DE LAS CAMPAÑAS ANTI-FUMAR?"
prompt = build_prompt(query)

print("Prompt generado para el LLM:")
print("="*60)
print(prompt)

Prompt generado para el LLM:

Responde con base en el siguiente contexto del negocio:

-------------------------------------------------------

cada vez más su amigo. 
 
D. EL EFECTO DE LAS CAMPAÑAS ANTI-FUMAR. 
 
El efecto de estas campañas es que ciega a los Fumadores del verdadero 
problema. Ellos se concentran en decirle a los Fumadores porqué no deben 
fumar, y ese no es el problema. Sabemos que no deberíamos fumar, pero no 
fumamos por esas razones, sin por otras. Ahora, quiero que se metan en la 
cabeza: 
 
EL ÚNICO PROBLEMA ES ¡REMOVER LAS RAZONES ILUSORIAS DE 
PORQUÉ FUMAMOS!
---
las terribles desventajas de ser un fumador y de buscar por excusas para 
fumar un cigarrito más. Tú no decides que nunca dejarás el cigarro, pero 
pospones el bendito día indefinidamente. Mientras más la sociedad intente 
asustar o humillar a los Fumadores, ellos encontrarán más y más se cerrarán, 
y los que hacen campaña serán sus enemigos, y el cigarro aparentará ser 
cada vez más su amigo. 
 
D. E

### LLM is Responding

In [22]:
model_path = "./gguf_gemma_model/gemma-2b-it.Q4_K_M.gguf"
if os.path.exists(pdf_path):
    print(f"File found at: {model_path}")
else:
    print(f"File not found at: {model_path}")

llm = Llama(model_path=model_path, n_ctx=4096, n_threads=8)

output = llm(prompt, max_tokens=None, stop=["Usuario:", "Sistema:"])
print(output["choices"][0]["text"])

llama_model_load_from_file_impl: using device Metal (AMD Radeon Pro 5500M) - 3751 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 164 tensors from ./gguf_gemma_model/gemma-2b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_cou

File found at: ./gguf_gemma_model/gemma-2b-it.Q4_K_M.gguf


llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,256000]  = ["<pad>", "<eos>", "<bos>", "<unk>", ...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,256000]  = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22: 

**El efecto de estas campañas es que ciega a los Fumadores del verdadero problema.**
